In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [3]:
word2ix = {"hello": 0, "world": 1}
embeds = nn.Embedding(2, 5) # 2 in vocab  5 dimension
lookup_tensor = torch.tensor(word2ix["hello"], dtype=torch.long)
hello_embed = embeds(lookup_tensor)
print(hello_embed)

tensor([-0.8923, -0.0583, -0.1955, -0.9656,  0.4224])


In [4]:
# constants defining
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10
# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()

In [5]:
trigrams = [([test_sentence[i], test_sentence[i+1]], test_sentence[i+2]) \
            for i in range(len(test_sentence)-2)]
print(trigrams[:3])

vocab = set(test_sentence)
word2ix = {word: i for i, word in enumerate(vocab)}

#class NGramLanguageModeler(nn.Module):
#    def __init__(self, )

[(['When', 'forty'], 'winters'), (['forty', 'winters'], 'shall'), (['winters', 'shall'], 'besiege')]


In [6]:
class NGramLanguageModeler(nn.Module):
    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)
        
    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_prob = F.log_softmax(out, dim=1)
        return log_prob

In [9]:
losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)f

In [18]:
for epoch in range(10):
    # total_loss = torch.tensor(word_to_ix[w] for w in context, dtype=torch.long)
    total_loss = torch.Tensor([0])
    for context, target in trigrams:
        context_idxs = torch.tensor([word2ix[w] for w in context], dtype=torch.long)
        model.zero_grad()
        log_prob = model(context_idxs)
        loss = loss_function(log_prob, torch.tensor([word2ix[target]], dtype=torch.long))
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    losses.append(total_loss)
print(losses)

[tensor([ 520.6190]), tensor([ 518.3424]), tensor([ 516.0795]), tensor([ 513.8294]), tensor([ 511.5901]), tensor([ 509.3613]), tensor([ 507.1431]), tensor([ 504.9327]), tensor([ 502.7309]), tensor([ 500.5369]), tensor([ 498.3511]), tensor([ 496.1713]), tensor([ 493.9974]), tensor([ 491.8306]), tensor([ 489.6686]), tensor([ 487.5113]), tensor([ 485.3580]), tensor([ 483.2070]), tensor([ 481.0585]), tensor([ 478.9119]), tensor([ 476.7672]), tensor([ 474.6238]), tensor([ 472.4803]), tensor([ 470.3364]), tensor([ 468.1929]), tensor([ 466.0487]), tensor([ 463.9028]), tensor([ 461.7560]), tensor([ 459.6072]), tensor([ 457.4579]), tensor([ 455.3063]), tensor([ 453.1522]), tensor([ 450.9952]), tensor([ 448.8331]), tensor([ 446.6664]), tensor([ 444.4960]), tensor([ 442.3214]), tensor([ 440.1410]), tensor([ 437.9544]), tensor([ 435.7612])]


In [19]:
# doing cbow
CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right
raw_text = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells.""".split()

# By deriving a set from `raw_text`, we deduplicate the array
vocab = set(raw_text)
vocab_size = len(vocab)

word_to_ix = {word: i for i, word in enumerate(vocab)}
data = []
for i in range(2, len(raw_text) - 2):
    context = [raw_text[i - 2], raw_text[i - 1],
               raw_text[i + 1], raw_text[i + 2]]
    target = raw_text[i]
    data.append((context, target))
print(data[:5])


[(['We', 'are', 'to', 'study'], 'about'), (['are', 'about', 'study', 'the'], 'to'), (['about', 'to', 'the', 'idea'], 'study'), (['to', 'study', 'idea', 'of'], 'the'), (['study', 'the', 'of', 'a'], 'idea')]


In [ ]:
# TODO: implement the CBOW module
class CBOW(nn.Module):

    def __init__(self):
        pass

    def forward(self, inputs):
        pass

# create your model and train.  here are some functions to help you make
# the data ready for use by your module
def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)


make_context_vector(data[0][0], word_to_ix)  # example

# TODO:CBOW train loop